In [ ]:
# default_exp teamstrength

# Team strength

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
import os
import dotenv

In [ ]:
#export
import abc
import datetime as dt

import mezzala

import wingback.db

In [ ]:
#export


class ModelABC:
    @abc.abstractmethod
    def fetch_data(self, league_ids, date):
        training_data = ...  # e.g. matches up-to, not including `date`
        test_data = ...      # matches on `date`
        return training_data, test_data
    
    @abc.abstractmethod
    def fit(self, data):
        return self
    
    @abc.abstractmethod
    def predict(self, data):
        predictions = ...
        return predictions

In [ ]:
#export


class DCGoals(ModelABC):
    def __init__(self, time_window=dt.timedelta(days=360), epsilon=-0.0065):
        self.time_window = time_window
        self._model = mezzala.DixonColes(
            adapter=mezzala.KeyAdapter(
                home_goals='home_goals',
                away_goals='away_goals',
                home_team='home_team_id',  # Might be nicer to do a tuple of (ID, name)?
                away_team='away_team_id',
            ),
            blocks=[
                mezzala.blocks.BaseRate(),
                mezzala.blocks.TeamStrength(),
                mezzala.blocks.HomeAdvantage(),
            ],
            weight=mezzala.weights.ExponentialWeight(
                epsilon=epsilon,  
                key=lambda x: x['days_ago']
            )
        )
    
    def fetch_data(self, league_ids, date):
        training_data = wingback.db.queries.fetch_matches(
            start=date-self.time_window, 
            end=date, 
            league_ids=league_ids,
            season_ids=[None]
        )
        test_data = wingback.db.queries.fetch_matches(
            start=date, 
            end=date+dt.timedelta(days=1), 
            league_ids=league_ids,
            season_ids=[None]
        )
        return list(training_data), list(test_data)
    
    def fit(self, data):
        self._model.fit(data)
        return self
    
    def predict(self, data):
        predictions = self._model.predict(data)
        return predictions

In [ ]:
dotenv.load_dotenv()

wingback.db.queries.connect('postgresql://{user}:{password}@{host}:{port}/{database}'.format(
    host=os.environ['DB_HOST'],
    user=os.environ['DB_USER'],
    password=os.environ['DB_PASS'],
    database=os.environ['DB_NAME'],
    port=os.environ['DB_PORT'],
))

In [ ]:
dc_model = DCGoals()

train, test = dc_model.fetch_data(
    league_ids=[1]
    dt.datetime(2021, 1, 1).date()
)

dc_model.fit(train)

predictions = dc_model.predict(test)

print(f'Home team: {test[0]["home_team"]}')
print(f'Away team: {test[0]["away_team"]}')
for outcome, prediction in mezzala.scorelines_to_outcomes(predictions[0]).items():
    print(f'{outcome.value.ljust(9)}: {prediction.probability:0.2f}')

In [ ]:
#export


class DCxG(ModelABC):
    def __init__(self):
        self._model = mezzala.DixonColes(
            adapter=...,
            blocks=[
                mezzala.blocks.BaseRate(),
                mezzala.blocks.TeamStrength(),
                mezzala.blocks.HomeAdvantage(),
            ],
            weight=mezzala.weights.KeyWeight(lambda x: x['probability'])
        )
    
    def fetch_data(self, date):
        training_data = ...  # e.g. matches up-to, not including `date`
        test_data = ...      # matches on `date`
        return training_data, test_data
    
    def fit(self, data):
        self._model.fit(data)
        return self
    
    def predict(self, data):
        base_predictions = self._model.predict(data)
        predictions = ... # flatten? add metadata?
        return predictions